# README


**Environment Setup and Data Loading:**

Checks for GPU availability and falls back to CPU if a GPU is not found.
Loads data from a CSV file (Bill of Materials or BOM data) into a pandas DataFrame. It checks if the file exists and whether the DataFrame is empty.

**Data Preprocessing:**

The function preprocess_bom_data performs label encoding on categorical columns and scales the numerical data using MinMaxScaler.

**GAN Model Creation:**

Two functions, make_generator_model and make_discriminator_model, define the architecture for the generator and discriminator parts of the GAN.
The generator model aims to generate new data, and the discriminator model tries to distinguish between real and generated data.

**GAN Training Functions:**

The function train_step performs one training step for the GAN, involving the generator and discriminator's training.
The function train_gan orchestrates the overall training process, creating the generator and discriminator, setting up loss functions and optimizers, and running the training loop for a specified number of epochs.

**Data Generation:**

After training, the script uses the trained generator model to create new data samples. This is done in the generate_new_data function.

**Postprocessing Generated Data:**

The function postprocess_generated_data postprocesses the generated data to ensure it adheres to certain constraints like unique part numbers and consistent units of measure for similar descriptions.

**Execution:**

The script prompts the user to enter the number of samples to generate.
Generated data is then postprocessed and printed.

**Key Aspects:**

TensorFlow and Keras are used for model definition and training.
Pandas and scikit-learn are used for data handling and preprocessing.
The script is designed to work with a specific type of data (BOM), but the principles could be adapted for other data types.

In [5]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import AdamW
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import numpy as np
import os
import random

# Check for GPU
if not tf.test.gpu_device_name():
    print('No GPU found. Using CPU')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

# Load BOM data
file_path = '/content/Raw-Material-Acqusition_BOM_.csv'  # please provide more samples of data so that generated data will be more meaninful.
if not os.path.exists(file_path):
    print(f"File not found: {file_path}")
    exit()  # Exit if file not found
else:
    bom_data = pd.read_csv(file_path)

    if bom_data.empty:
        print("The DataFrame is empty. Please check the CSV file content.")
        exit()  # Exit if data is empty
    else:
        print("Data loaded successfully.")

# Data Preprocessing with Label Encoding
def preprocess_bom_data(data):
    label_encoders = {}
    for column in data.columns:
        if data[column].dtype == 'object':
            le = LabelEncoder()
            data[column] = le.fit_transform(data[column])
            label_encoders[column] = le

    # Scale your data
    scaler = MinMaxScaler()
    data_scaled = scaler.fit_transform(data)
    return data_scaled, scaler, label_encoders

data_scaled, scaler, label_encoders = preprocess_bom_data(bom_data)

# The number of columns in the original data
original_feature_count = bom_data.shape[1]

# GAN Model Definitions
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(128, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(256, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dense(original_feature_count, activation='relu'))
    return model

def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(256, input_shape=(original_feature_count,)))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(128))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# GAN Training Functions
@tf.function
def train_step(images, batch_size, generator, discriminator, cross_entropy, generator_optimizer, discriminator_optimizer):
    noise = tf.random.normal([batch_size, 100])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
        disc_loss = cross_entropy(tf.ones_like(real_output), real_output) + cross_entropy(tf.zeros_like(fake_output), fake_output)
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

def train_gan(data, epochs, batch_size):
    generator = make_generator_model()
    discriminator = make_discriminator_model()
    cross_entropy = tf.keras.losses.BinaryCrossentropy()
    generator_optimizer = AdamW(learning_rate=1e-4)
    discriminator_optimizer = AdamW(learning_rate=1e-4)
    data = tf.data.Dataset.from_tensor_slices(data).batch(batch_size)
    for epoch in range(epochs):
        for image_batch in data:
            train_step(image_batch, batch_size, generator, discriminator, cross_entropy, generator_optimizer, discriminator_optimizer)
    return generator

# Train the GAN
generator = train_gan(data_scaled, epochs=50, batch_size=32)

# Data Generation (using original_feature_count)
def generate_new_data(generator, num_samples):
    noise = tf.random.normal([num_samples, 100])
    generated_data = generator(noise, training=False)
    return pd.DataFrame(generated_data.numpy(), columns=bom_data.columns)

# Postprocess Generated Data
def postprocess_generated_data(generated_data, label_encoders, part_number_digits=6):
    # Ensuring unique part numbers
    unique_part_numbers = set()
    min_part_number = 10**(part_number_digits - 1)
    max_part_number = (10**part_number_digits) - 1
    for index in range(len(generated_data)):
        while True:
            part_num = random.randint(min_part_number, max_part_number)
            if part_num not in unique_part_numbers:
                unique_part_numbers.add(part_num)
                generated_data.at[index, 'Part Number'] = part_num
                break

    # Ensuring same description has same unit of measure and varying prices
    description_unit_map = {}
    for index, row in generated_data.iterrows():
        description = row['Description']
        if description in description_unit_map:
            generated_data.at[index, 'Unit of Measure'] = description_unit_map[description]['unit']
            # Apply variation to unit price
            base_price = description_unit_map[description]['price']
            variation = random.uniform(-0.05 * base_price, 0.05 * base_price)
            generated_data.at[index, 'Unit Price'] = base_price + variation
        else:
            description_unit_map[description] = {'unit': row['Unit of Measure'], 'price': row['Unit Price']}

    # Processing other columns
    for column in generated_data.columns:
        if column in label_encoders and column != 'Part Number':
            # Convert continuous data back to categorical
            generated_data[column] = generated_data[column].round().astype(int)
            le = label_encoders[column]
            generated_data[column] = le.inverse_transform(generated_data[column].clip(0, len(le.classes_) - 1))
        else:
            # Round to 2 decimal places for prices
            if column == 'Unit Price':
                generated_data[column] = generated_data[column].round(2)
            else:
                generated_data[column] = generated_data[column].astype(int)

    return generated_data

# Generate new data
num_samples = int(input("Enter the number of samples to generate: "))
generated_data = generate_new_data(generator, num_samples)

# Postprocess the generated data
processed_data = postprocess_generated_data(generated_data, label_encoders)

# Print the postprocessed generated data
print(processed_data)


No GPU found. Using CPU
Data loaded successfully.
Enter the number of samples to generate: 20
    Part Number      Description Unit of Measure Quantity Unit Price  \
0        211432  Silicon dioxide            Each    1,000      $0.10   
1        925375  Silicon nitride            Each    1,000      $0.10   
2        747088  Silicon dioxide            Each    1,000      $0.10   
3        226127  Silicon dioxide            Gram    1,000      $0.10   
4        416333  Silicon dioxide            Each      100      $0.10   
5        712327  Silicon dioxide            Each    1,000      $0.10   
6        961164  Silicon dioxide            Each    1,000      $0.10   
7        189136  Silicon dioxide            Each    1,000      $0.10   
8        513313  Silicon dioxide            Each    1,000      $0.10   
9        503793  Silicon dioxide            Each    1,000      $0.10   
10       172492  Silicon nitride            Each      100     $20.00   
11       445566  Silicon dioxide          